# WB

In [ ]:
import pandas as pd
import geopandas as gpd

gdf_full = gpd.read_feather(r"D:\World Bank\CLIENT v2\Data\Data_proc\WB_country_IDs.feather")

In [ ]:
# Generados Nico
gdf = gdf_full.drop(columns=["OBJECTID", "geometry"])
gdf.columns = gdf.columns.str.lower()
gdf = gdf.rename(columns={"id":"ID"})

for shock in ["drought", "floods", "hurricanes", "intenserain", "heatwaves", "coldwaves"]:
    df = pd.read_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\WB_{shock}_long.csv")

    dflong = df.drop(columns=["Unnamed: 0", "chunk"]).melt(id_vars=["ID", "year", "variable", "threshold"], var_name="measure", value_name="value")
    print(dflong.measure.unique())
    dflong = gdf.merge(dflong, on="ID", validate="1:m").drop(columns=["ID"]) 
    dflong.to_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_{shock}.csv", index=False)

In [ ]:
# Mapas
gdf_full.drop(columns=["OBJECTID", "ID"]).to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_map.csv", index=False)

# IPUMS

In [1]:
import pandas as pd
import geopandas as gpd

gdf_full = gpd.read_feather(r"D:\World Bank\CLIENT v2\Data\Data_proc\IPUMS_country_IDs.feather")

In [ ]:
# Mapas
gdf_full = gdf_full.drop(columns=["ID"])
gdf_full = gdf_full.rename(columns={"CNTRY_CODE":"adm0", "GEOLEVEL1":"adm1", "GEOLEVEL2":"adm2"})
gdf_full[["adm0_name", "adm1_name", "adm2_name"]] = "To be filled"
gdf_full.to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\IPUMS_map.csv", index=False)

In [53]:
import os

# Shocks
path = r"D:\World Bank\CLIENT v2\Data\Data_out\HC Treatment Complete"
       
files = os.listdir(path)
files = [f for f in files if "HC_national_data" in f and f.endswith(".csv")]

dfs = []
for file in files:
    df = pd.read_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\HC Treatment Complete\{file}")
    df["s3"] = pd.NA
    df["s4"] = pd.NA
    
    s3cols = ["s3a", "s3b", "s3c", "s3d", "s3e", "s3f"]
    s4cols = ["s4a", "s4b", "s4c"]

    for col in s3cols: 
        df["s3"] = df["s3"].fillna(df[col])
        assert (df[s3cols].notna().sum(axis=1) <= 1).all(), f"{df[(df[s3cols].notna().sum(axis=1) <= 1)]}"
    for col in s4cols:
        df["s4"] = df["s4"].fillna(df[col])
        assert (df[s4cols].notna().sum(axis=1) <= 1).all()

    dfs += [df]
    
df = pd.concat(dfs)
for col in df.columns:
    assert df[col].isna().all() == False

# Drop s3* columns
df = df.drop(columns=[col for col in df.columns if ("s3" in col or "s4" in col) and (col != "s3" and col != "s4")])
# Order variables
df = df[["adm0", "s1", "s2", "s3", "s4", "s5", "outcome", "new", "v"]]
print(f"Hay datos de {df.adm0.unique().size} países")

C:\Users\ofici\AppData\Local\Temp\ipykernel_9684\1458644849.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["s3"] = df["s3"].fillna(df[col])
C:\Users\ofici\AppData\Local\Temp\ipykernel_9684\1458644849.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["s4"] = df["s4"].fillna(df[col])
C:\Users\ofici\AppData\Local\Temp\ipykernel_9684\1458644849.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future

AssertionError: 

In [51]:
df[(df[[col for col in df.columns if "s3" in col]].notna().sum(axis=1) ==2)]

,adm0,new,outcome,s1,s2,s3a,s3b,s3c,s3d,s3e,s3f,s4a,s4b,s4c,s5,status,v,s3,s4
0,32,Before,Child count: z-score,Hurricane,Area,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,10,0,Not-affected,-0.198146,3.0,<NA>
1,32,Before,Completed primary education (ages 15-24),Hurricane,Area,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,10,0,Not-affected,0.864746,3.0,<NA>
2,32,After,Completed primary education (ages 15-24),Hurricane,Area,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,10,0,Not-affected,0.891553,3.0,<NA>
3,32,Before,Completed primary education (ages 25+),Hurricane,Area,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,10,0,Not-affected,0.744479,3.0,<NA>
4,32,After,Completed primary education (ages 25+),Hurricane,Area,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,10,0,Not-affected,0.813061,3.0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316794,894,After,School attendance (ages 6-14),Hurricane,Population,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,50,50,Not-affected,1.000000,5.0,<NA>
316795,894,Before,Water inside dwelling,Hurricane,Population,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,50,50,Not-affected,0.072123,5.0,<NA>
316796,894,After,Water inside dwelling,Hurricane,Population,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,50,50,Not-affected,0.003554,5.0,<NA>
316797,894,Before,Years of education (ages 15-24),Hurricane,Population,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,50,50,Not-affected,4.530945,5.0,<NA>


In [42]:
df

,adm0,s1,s2,s3,s4,s5,outcome,new,v
0,32,Hurricane,Area,3.0,10.0,0,Child count: z-score,Before,-0.198146
1,32,Hurricane,Area,3.0,10.0,0,Completed primary education (ages 15-24),Before,0.864746
2,32,Hurricane,Area,3.0,10.0,0,Completed primary education (ages 15-24),After,0.891553
3,32,Hurricane,Area,3.0,10.0,0,Completed primary education (ages 25+),Before,0.744479
4,32,Hurricane,Area,3.0,10.0,0,Completed primary education (ages 25+),After,0.813061
...,...,...,...,...,...,...,...,...,...
1264815,894,Drought,Population,spi,2.5,50,School attendance (ages 6-14),After,1.000000
1264816,894,Drought,Population,spi,2.5,50,Water inside dwelling,Before,0.072123
1264817,894,Drought,Population,spi,2.5,50,Water inside dwelling,After,0.003554
1264818,894,Drought,Population,spi,2.5,50,Years of education (ages 15-24),Before,4.530945


In [46]:
(df[[col for col in df.columns if "s3" in col]].notna().sum(axis=1) == 1).all()

False

In [10]:
merged = df.merge(gdf_full[["CNTRY_CODE"]].drop_duplicates(), left_on="adm0", right_on="CNTRY_CODE", how="outer", indicator=True)
merged[merged._merge == "right_only"].CNTRY_CODE.unique()

array([ 32.,  40.,  50.,  68.,  76., 231., 238., 239., 275., 276., 356.,
       368., 376., 504., 566., 586., 662.])

In [11]:
merged._merge.value_counts()

_merge
both          1831927
right_only         17
left_only           0
Name: count, dtype: int64